In [1]:
!ls ../model/

glove.pt      gloveMixed.pt


In [2]:
import torch
import glove
import run
import pickle
from tools import Dictionary

In [3]:
EMBEDDING_SIZE = 128
CONTEXT_SIZE = 3
NUM_EPOCH = 100
BATHC_SIZE = 512
PATH = "../model/gloveMixed.pt"

In [4]:
with open("data/f.pkl", mode='rb') as fp:
    doc = pickle.load(fp)

In [5]:
dictionary = Dictionary()
dictionary.update(doc)
# doc.vocab_size
VS = dictionary.vocab_size

In [6]:
len(list(dictionary.word2idx.keys()))

21281

In [7]:
model = glove.GloVeMixedCurvature(EMBEDDING_SIZE, CONTEXT_SIZE, VS)
model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))
model

GloVeMixedCurvature(
  (euc): ManifoldEmbedding(
    (_focal_embeddings): Embedding(21281, 128)
    (_context_embeddings): Embedding(21281, 128)
    (_focal_biases): Embedding(21281, 1)
    (_context_biases): Embedding(21281, 1)
    (manifold): Stereographic manifold
  )
  (hyp): ManifoldEmbedding(
    (_focal_embeddings): Embedding(21281, 128)
    (_context_embeddings): Embedding(21281, 128)
    (_focal_biases): Embedding(21281, 1)
    (_context_biases): Embedding(21281, 1)
    (manifold): Stereographic manifold
  )
  (sph): ManifoldEmbedding(
    (_focal_embeddings): Embedding(21281, 128)
    (_context_embeddings): Embedding(21281, 128)
    (_focal_biases): Embedding(21281, 1)
    (_context_biases): Embedding(21281, 1)
    (manifold): Stereographic manifold
  )
  (_focal_embeddings): Embedding(21281, 128)
  (_context_embeddings): Embedding(21281, 128)
  (_focal_biases): Embedding(21281, 1)
  (_context_biases): Embedding(21281, 1)
)

In [8]:
ws = torch.nn.functional.softmax(model.w)

curvatures = {
    'euc': float(model.euc.manifold.k.data.detach().numpy()),
    'hyp': float(model.hyp.manifold.k.data.detach().numpy()),
    'sph': float(model.sph.manifold.k.data.detach().numpy()),
} 

weights = {
    'euc': ws.detach().numpy()[0],
    'hyp': ws.detach().numpy()[1],
    'sph': ws.detach().numpy()[2],
} 

print(curvatures)
print(weights)

{'euc': 0.058570027351379395, 'hyp': 1.0, 'sph': 1.0}
{'euc': 0.9999939, 'hyp': 5.7713128e-06, 'sph': 3.5080123e-07}


/var/folders/s5/hdjj6v6x4x5crg2lctj_26cr0000gn/T/ipykernel_78349/2681497438.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  ws = torch.nn.functional.softmax(model.w)


In [ ]:
emb = [
    model.euc._focal_embeddings.weight + model.euc._context_embeddings.weight,
    model.hyp._focal_embeddings.weight + model.hyp._context_embeddings.weight,
    model.sph._focal_embeddings.weight + model.sph._context_embeddings.weight,
]
emb = [e.detach().numpy() for e in emb]
emb = dict(zip(emb, ["euc", "hyp"]))

In [ ]:
emb[0].shape

In [ ]:
mixedGlove = {}
for w in dictionary.word2idx:
    mixedGlove[w] = {
        "euc":emb[0][dictionary.word2idx[w]],
        "hyp":emb[1][dictionary.word2idx[w]],
        "sph":emb[2][dictionary.word2idx[w]],
    }

In [ ]:
embedding = {
    "curvatures": curvatures,
    "weights": weights,
    "vecs": mixedGlove,
}

In [ ]:
# embedding

In [ ]:
with open("mixed.pkl", "wb") as handle:
    pickle.dump(embedding, handle)